In [ ]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil
import glob

%matplotlib inline 

In [ ]:
def conv(val):
    if val == np.nan:
        return 0
    return val

In [ ]:
df = pd.read_csv("../../diaries_transactions_all.csv", low_memory=False);


### Poverty file

In [ ]:
dfp = pd.read_csv("../../pov_hh_over_trx_yr_mo_kes_excRR.csv", low_memory=False);
dfp.head()


In [ ]:
dfp.shape

In [ ]:
dfp.describe()

### Here we check poverty info with housing conditions

In [ ]:
dfh = pd.read_csv('../../housing_conditions_ownership.csv')

In [ ]:
dfh.head()

In [ ]:
ids_pov = set(dfp["hh_ids"].unique())
len(ids_pov)

In [ ]:
ids_hou = set(dfh["hh_ids"].unique())
len(ids_hou)

In [ ]:
if (ids_pov & ids_hou):
    print(len(ids_pov & ids_hou))
else:
    print("no matches")

In [ ]:
for ids in ids_pov:
    if ids in ids_hou:
        continue
    else: print(ids)

### Check matches for the remittances as well 


In [ ]:
dfr= pd.read_csv("../../rem_ind_over_trx_yr_mo_kes.csv")
dfr.head()


In [ ]:
dfr["m_ids_owner"].describe()

In [ ]:
dfr["hh_ids"].describe()

In [ ]:
ids_rem = dfr["hh_ids"].unique()


In [ ]:
for ids in ids_rem:
    if ids in ids_pov:
        continue
    else: print(ids)

In [ ]:
ids_rem = set(dfr["m_ids_owner"].unique())
ids_rem.remove("HH")
dfr["m_ids_owner"].value_counts()

In [ ]:
(dfr["m_ids_owner"].value_counts()>1).value_counts()

### Here we are interested in hh size, monthly consumption and income 



  WE can assess whether family has enough money to cover their needs or they rely on help of relatives and friends most of the time. If monthly left is negative, then family is in debt and they are unable to take any other financil burden as they will not be able to pay it off. 

In [ ]:
dfp['monthly_left'] = dfp.apply(lambda row: row.inc_hh_medianpti - row.con_hh_medpti, axis = 1) 

In [ ]:
dfp[['hh_ids', 'inc_hh_medianpti', 'dem_hh_size', 'con_hh_medpti', 'monthly_left']]


### Here is another file with RR included. Not a big sifference, so I assume we can exclude RR and see if the famili is not in debt without RR

In [ ]:
dfp2 = pd.read_csv("../../pov_hh_over_trx_yr_mo_kes_incRR.csv", low_memory=False);
dfp2.head()

In [ ]:
dfp2['monthly_left'] = dfp.apply(lambda row: row.inc_hh_medianpti - row.con_hh_medpti, axis = 1) 
dfp2[['hh_ids', 'inc_hh_medianpti', 'dem_hh_size', 'con_hh_medpti', 'monthly_left']]


In [ ]:
dfp['monthly_left'].equals(dfp2['monthly_left'])

In [ ]:
dfh.drop(['hh_int_situation', 'hh_circ_move_why', 'hh_circ_move_when', 'hh_asset_fryingpan_N' ], axis=1, inplace=True)

In [ ]:
dfh.columns


In [ ]:
hou_dum = pd.get_dummies(data=dfh, columns=['hh_inf_wall_material', 'hh_circ_problems', 'hh_inf_roof_material','hh_inf_floor_material','hh_inf_cookfuel_source', 'hh_inf_lighting_source', 'hh_inf_water_source', 'hh_inf_toilet_type', 'hh_inf_toilet_shared', 'hh_inf_housing_type',
       'hh_inf_electricity_access', 'hh_inf_interiorcond',
       'hh_inf_exteriorcond', 'hh_asset_internet_access', 'hh_asset_iron_any',
       'hh_asset_mosqnets_N', 'hh_asset_towels_N', 'hh_env_settingdesc',
       'hh_env_desc', 'hh_env_dilapbuildings', 'hh_env_streets',
       'hh_env_trash', 'hh_ownership_type', 'hh_ownership_plotacquisition',
       'hh_ownership_houseacquisition', 'hh_ownership_docu', 'hh_otherprop_any',  'hh_otherprop1_use', 'hh_otherprop2_use','hh_otherprop2_docu', 'hh_otherprop3_docu','hh_otherprop3_use', 'hh_othershamba1_docu', 'hh_othershamba1_use', 'hh_othershamba2_docu', 'hh_othershamba2_use','hh_othershamba3_docu', 'hh_othershamba3_use' ])

In [ ]:
hou_dum.describe

In [ ]:
pov_hh = pd.read_csv('../../pov_hh_over_trx_yr_mo_kes_excRR.csv', dtype = {'hh_ids': 'object'})
trx = pd.read_csv('../../diaries_transactions_all.csv',dtype = {'hh_ids': 'object'}, low_memory = False)

In [ ]:
keep_ids = list(trx.m_ids_owner.unique())

In [ ]:
pov_hh.head()

In [ ]:
poverty = dfp.merge(df, how = 'left', on='hh_ids')

In [ ]:
poverty.shape

In [ ]:
pov_hou= hou_dum.merge(poverty, how = 'left', on = 'hh_ids')

In [ ]:
pov_hou.shape

In [ ]:
pov_hou_rem = dfr.merge(pov_hou, how = 'left', on = 'hh_ids')

In [ ]:
pov_hou_rem.shape

In [ ]:
pov_hou_rem.to_csv('pov_hou_rem_merge.csv')